In [40]:
import pandas as pd

In [41]:
df = pd.read_csv('raw_data.csv')

In [48]:
print(df[df['price'].isna()])

       zip_code          commune         province  type_of_property  \
561        8500         Kortrijk  West-Vlaanderen                 0   
2884       6700         Toernich       Luxembourg                 0   
3354       9300            Aalst  Oost-Vlaanderen                 0   
3911       8540         Deerlijk  West-Vlaanderen                 0   
4385       6200      Châtelineau          Hainaut                 0   
4670       6791            Athus       Luxembourg                 0   
4945       2500             Lier        Antwerpen                 0   
4980       6000        Charleroi          Hainaut                 0   
5556       1030       Schaerbeek        Bruxelles                 0   
6575       7500          Tournai          Hainaut                 0   
7311       8370         Uitkerke  West-Vlaanderen                 0   
12305      4000          Rocourt            Liège                 0   
16965      9620         Zottegem  Oost-Vlaanderen                 1   
17475 

In [51]:
print(df.price.isna().sum())
print(dfc.price.isna().sum())

dfc = df.dropna(subset=['price'], how='all')
dfc.reset_index(inplace=True)

36
0


In [ ]:
dfc.drop(['index'], axis=1, inplace=True)
print(dfc)
#

In [58]:
dfc.to_csv('cleaned-data.csv', index=False)